# Preparing ARIA Sentinel-1 data for validation of Solid Earth requirements

**Original code authored by:** David Bekaert, Heresh Fattahi, Eric Fielding, and Zhang Yunjun  <br>
Extensive modifications by Adrian Borsa and Amy Whetter 2022 <br>
Reorganized and modified by Ekaterina Tymofyeyeva, March 2024 <br>
Clean up and new functionality by Emre Havazli, April 2025

<div class="alert alert-warning">
This notebook pre-processes data for different NISAR Solid Earth calval sites amd requirements. Subsequent validation is done via separate notebooks for the Transient, Secular, and Coseismic requirements. These are located under /ATBD_main/methods/.
</div>

<hr/>

## Table of Contents: <a id='prep_TOC'></a>

[**Environment Setup**](#setup)
- [Load Python Packages](#load_packages)
- [Define CalVal Site and Parameters](#set_calval_params)
- [Define Directories](#set_directories)
- [Authentication](#set_authentication)

[**1. Download and Prepare Interferograms**](#prep_ifg)
- [1.1.  Download Interferograms](#prep_download_ifg)
- [1.2.  Crop Interferograms](#prep_crop_ifg)
- [1.3.  Set Up MintPy Configuration file](#prep_setup_config)
- [1.4.  Load Data into MintPy](#prep_load_data)

<hr/>

<a id='#setup'></a>
## Environment Setup

### Load Python Packages <a id='#load_packages'></a>

In [ ]:
import json
import netrc
import os
import subprocess
import glob

### Define Calval Site and Parameters <a id='set_calval_params'></a>

In [ ]:
# === Basic Configuration ===
site = "CalVal_S1_LosAngelesA64"  # Cal/Val location ID
requirement = "Secular"  # Options: 'Secular', 'Coseismic', 'Transient'
dataset = "ARIA_S1_new"  # Dataset type: 'ARIA_S1', 'ARIA_S1_new'
aria_gunw_version = "3_0_1"

rundate = "20250818"  # Date of this Cal/Val run
version = "1"         # Version of this Cal/Val run
custom_sites = "/home/jovyan/my_sites.txt"  # Path to custom site metadata

# === Username Detection / Creation ===
user_file = "/home/jovyan/me.txt"
if os.path.exists(user_file):
    with open(user_file, "r") as f:
        you = f.readline().strip()
else:
    you = input("Please type a username for your Cal/Val outputs: ").strip()
    with open(user_file, "w") as f:
        f.write(you)

# === Load Cal/Val Site Metadata ===
try:
    with open(custom_sites, "r") as f:
        sitedata = json.load(f)
    site_info = sitedata["sites"][site]
except (FileNotFoundError, json.JSONDecodeError) as e:
    raise RuntimeError(f"Failed to load site metadata from {custom_sites}: {e}")
except KeyError:
    raise ValueError(f"Site ID '{site}' not found in {custom_sites}")

print(f"Loaded site: {site}")

### Set Directories and Files <a id='set_directories'></a>

In [ ]:
# Static base directory for Cal/Val processing
BASE_DIR = "/scratch/nisar-st-calval-solidearth"

# Define key path components
site_dir = os.path.join(BASE_DIR, dataset, site)
work_dir = os.path.join(site_dir, requirement, you, rundate, f"v{version}")
gunw_dir = os.path.join(site_dir, "products")
mintpy_dir = os.path.join(work_dir, "MintPy")

# Create required directories
for path in [work_dir, gunw_dir, mintpy_dir]:
    os.makedirs(path, exist_ok=True)

# Set working directory
os.chdir(work_dir)

# Log directory structure
print(f"  Work directory: {work_dir}")
print(f"  GUNW directory: {gunw_dir}")
print(f"MintPy directory: {mintpy_dir}")

# Configuration file path
site_code = site_info.get('calval_location')
config_file = os.path.join(mintpy_dir, f"{site_code}.cfg")

### Authentication <a id='set_authentication'></a>

In [ ]:
def ensure_permission(path, mode=0o600):
    if os.path.exists(path):
        os.chmod(path, mode)

# === Earthdata Login ===
fnetrc = os.path.expanduser("~/.netrc")
earthdata_host = "urs.earthdata.nasa.gov"
earthdata = False

if os.path.exists(fnetrc):
    ensure_permission(fnetrc)
    nrc = netrc.netrc()
    credentials = nrc.authenticators(earthdata_host)
    if credentials:
        earthdata_user, _, earthdata_password = credentials
        earthdata = True
        print(f"Earthdata credentials found for user: {earthdata_user}")

if not earthdata:
    print("\nNEEDED to Download ARIA GUNWs")
    print("Create account at: https://urs.earthdata.nasa.gov/")
    earthdata_user = input("Earthdata username: ").strip()
    earthdata_password = input("Earthdata password: ").strip()
    with open(fnetrc, "a") as f:
        f.write(f"machine {earthdata_host}\nlogin {earthdata_user}\npassword {earthdata_password}\n")
    ensure_permission(fnetrc)
    print("Earthdata credentials saved.")


# === OpenTopography API Key ===
fopentopo = os.path.expanduser("~/.topoapi")
if os.path.exists(fopentopo):
    ensure_permission(fopentopo)
    with open(fopentopo) as f:
        opentopography_api_key = f.read().strip()
else:
    print("\nNEEDED To Download DEMs:")
    print("Register at: https://portal.opentopography.org/login")
    print("Navigate: My Account → myOpenTopo Authorizations and API Key → Request API key")
    opentopography_api_key = input("OpenTopo API key: ").strip()
    with open(fopentopo, "w") as f:
        f.write(opentopography_api_key + "\n")
    ensure_permission(fopentopo)
    print("OpenTopography API key saved.")

# === CDS (ERA5) API Key ===
cds_config_path = os.path.expanduser("~/.cdsapirc")
if not os.path.exists(cds_config_path):
    print("\nNEEDED to use ERA5 correction:")
    print("Register and get token: https://cds.climate.copernicus.eu/how-to-api")
    cds_key = input("CDS API key (uid:api-key): ").strip()
    with open(cds_config_path, "w") as f:
        f.write("url: https://cds.climate.copernicus.eu/api\n")
        f.write(f"key: {cds_key}\n")
    ensure_permission(cds_config_path)
    print("CDS API config created.")
else:
    print("CDS API config file detected. (Ensure it is current)")

<br>
<hr>

<a id='prep_ifg'></a>
## 1. Download and Prepare Interferograms

In this initial processing step, all the necessary Level-2 unwrapped interferogram products are gathered, organized and reduced to a common grid for analysis with MintPy. Ascending and descending stacks of nearest-neighbor and skip-1 interferograms will be prepared for independent analysis. We use the open-source ARIA-tools package to download processed L2 interferograms over selected cal/val regions from the Alaska Satellite Facility archive and to stitch/crop the frame-based NISAR GUNW products to stacks that can be directly ingested into MintPy for time-series processing. ARIA-tools uses a phase-minimization approach in the product overlap region to stitch the unwrapped and ionospheric phase, a mosaicing approach for coherence and amplitude, and extracts the geometric information from the 3D data cubes through a mosaicking of the 3D datacubes and subsequent intersection with a DEM.

REFERENCE: https://github.com/aria-tools/ARIA-tools

### 1.1. Download GUNW Interferograms <a id='prep_download_ifg'></a>

In [ ]:
print(f"CalVal site: {site}")

# Extract site-specific metadata
bbox = site_info.get('download_region')
startdate = site_info.get('download_start_date')
enddate = site_info.get('download_end_date')
track = site_info.get('sentinel_track')

# Base command template
aria_cmd_template = (
    "ariaDownload.py --num_threads 8 "
    "-b {bbox} -u {user} -p \"{password}\" "
    "-s {start} -e {end} -t {track} "
    "--workdir {workdir} --version {version} -o {output}"
)

# Define common formatting args
common_args = dict(
    bbox=bbox,
    start=startdate,
    end=enddate,
    track=track,
    version=aria_gunw_version,
    user=earthdata_user,
    password=earthdata_password,
    workdir=gunw_dir
)

# Step 1: Count available GUNW products
count_cmd = aria_cmd_template.format(**common_args, output="count")
subprocess.run(count_cmd, text=True, shell=True, check=True)

# Step 2: Generate URL list
url_cmd = aria_cmd_template.format(**common_args, output="Url")
subprocess.run(url_cmd, text=True, shell=True, check=True)

# Step 3: Download GUNW products
print("Starting GUNW download...")
download_cmd = aria_cmd_template.format(**common_args, output="Download")
subprocess.run(download_cmd, text=True, shell=True, check=True)
print("Finished GUNW download.")

# Cleanup unnecessary files
cleanup_files = ["avg_rates.csv", "ASFDataDload0.py", "AvgDlSpeed.png", "error.log"]
for filename in cleanup_files:
    for path in [gunw_dir, work_dir]:
        full_path = os.path.join(path, filename)
        if os.path.exists(full_path):
            print(f"Cleaning file {full_path}")
            os.remove(full_path)


### 1.2. Crop and Mask Interferograms <a id='prep_crop_ifg'></a>

In [ ]:
# Parse date range from site metadata
start_date = int(site_info.get('download_start_date'))
end_date = int(site_info.get('download_end_date'))

# Filter GUNW files based on date range and version
gunw_list = []
for filename in os.listdir(gunw_dir):
    if not filename.endswith(".nc"):
        continue
    if aria_gunw_version not in filename:
        continue
    try:
        date1 = int(filename[30:38])  # reference date
        date2 = int(filename[21:29])  # secondary date
        if start_date <= date1 and date2 <= end_date:
            gunw_list.append(os.path.join(gunw_dir, filename))
    except (ValueError, IndexError):
        print(f"Warning: Skipping malformed filename: {filename}")

# Sort and write list to product file
gunw_list.sort()
product_file = os.path.join(work_dir, "product_file.txt")
with open(product_file, "w") as f:
    f.write("\n".join(gunw_list))

print(f"Wrote {len(gunw_list)} GUNW files to: {product_file}")


In [ ]:
# Determine optional correction layers based on site metadata
optional_layers = []

layer_conditions = [
    ("solidEarthTide", site_info.get('do_SET') == 'True'),
    ("ionosphere", site_info.get('do_iono') == 'True'),
    ("troposphereTotal", (
        site_info.get('do_tropo') == 'True' and 
        site_info.get('tropo_model') == 'HRRR'
    ))
]

optional_layers = [layer for layer, condition in layer_conditions if condition]

print(f"Optional correction layers {optional_layers} will be extracted")

In [ ]:
# Move to working directory
os.chdir(work_dir)

# Check if stacks already exists and extract if not
stack_dir = os.path.join(work_dir, 'stack')
if not os.path.exists(stack_dir):
    print('Preparing GUNWs for MintPy...')

    # Construct base command
    cmd_parts = [
        "ariaTSsetup.py",
        f"-f {product_file}",
        f"-b {site_info.get('analysis_region')}",
        f"-l '{', '.join(optional_layers)}'",
        "--croptounion",
        "-nt 8",
        "--log-level info"
    ]

    # Add water mask option if enabled
    if site_info.get('maskWater') != 'False':
        cmd_parts.append("--mask Download")

    # Run ariaTSsetup.py
    subprocess.run(" ".join(cmd_parts), shell=True, text=True)

else:
    print("Stack directory detected and not overwritten.")


# Update mask file
mask_file = glob.glob(f"{work_dir}/mask/*.msk")
if len(mask_file) == 1:
    mask_file = mask_file[0]
else:
    mask_file = "auto"

print(f"Water mask file: {mask_file}")

### 1.3. Set Up MintPy Configuration file <a id='prep_setup_config'></a>

The default processing parameters for MintPy's **smallbaselineApp.py** need to be modified by including the following lines in config_file (which must be manually created and placed into mint_dir):

- mintpy.load.processor      = aria
- mintpy.compute.cluster     = local
- mintpy.compute.numWorker   = auto
- mintpy.load.unwFile        = ../stack/unwrapStack.vrt
- mintpy.load.corFile        = ../stack/cohStack.vrt
- mintpy.load.connCompFile   = ../stack/connCompStack.vrt
- mintpy.load.demFile        = ../DEM/SRTM_3arcsec.dem
- mintpy.load.incAngleFile   = ../incidenceAngle/{download_start_date}_{download_edn_date}.vrt
- mintpy.load.azAngleFile    = ../azimuthAngle/{download_start_date}_{download_edn_date}.vrt
- mintpy.load.waterMaskFile  = ../mask/watermask.msk
- mintpy.reference.lalo      = auto, or somewhere in your bounding box
- mintpy.topographicResidual.pixelwiseGeometry = no
- mintpy.troposphericDelay.method              = no
- mintpy.topographicResidual                   = no

In [ ]:
os.chdir(mintpy_dir)

# Build config as a dictionary first
config_file_content = {
    "mintpy.load.processor": "aria",
    "mintpy.compute.cluster": "local",
    "mintpy.compute.numWorker": "auto",
    "mintpy.load.unwFile": f"{work_dir}/stack/unwrapStack.vrt",
    "mintpy.load.corFile": f"{work_dir}/stack/cohStack.vrt",
    "mintpy.load.connCompFile": f"{work_dir}/stack/connCompStack.vrt",
    "mintpy.load.demFile": f"{work_dir}/DEM/glo_90.dem",
    "mintpy.load.incAngleFile": f"{work_dir}/incidenceAngle/*.vrt",
    "mintpy.load.azAngleFile": f"{work_dir}/azimuthAngle/*.vrt",
    "mintpy.load.waterMaskFile": mask_file,
    "mintpy.topographicResidual.pixelwiseGeometry": "no",
    "mintpy.troposphericDelay.method": "no",
    "mintpy.topographicResidual": "no",
    "mintpy.network.tempBaseMax": site_info.get('tempBaseMax'),
    "mintpy.network.startDate": site_info.get('download_start_date'),
    "mintpy.network.endDate": site_info.get('download_end_date'),
    "mintpy.velocity.startDate": site_info.get('download_start_date'),
    "mintpy.velocity.endDate": site_info.get('download_end_date'),
    "mintpy.reference.lalo": site_info.get('reference_lalo'),
    "mintpy.network.excludeDate12": site_info.get('ifgExcludePair'),
    "mintpy.network.excludeDate" : site_info.get('ifgExcludeDate'),
    "mintpy.network.excludeIfgIndex" : site_info.get('ifgExcludeIndex'),
}

# Write config dictionary to text file
with open(config_file, "w") as f:
    f.writelines(f"{k} = {v}\n" for k, v in config_file_content.items())

# Confirm output
print(f"MintPy config file written to:\n    {config_file}\n")
with open(config_file, "r") as f:
    print(f.read())


### 1.4. Load Data into MintPy Cubes <a id='prep_load_data'></a>

The output of this step is an "inputs" directory in 'calval_directory/calval_location/MintPy/" containing two HDF5 files:
- ifgramStack.h5: This file contains 6 dataset cubes (e.g. unwrapped phase, coherence, connected components etc.) and multiple metadata
- geometryGeo.h5: This file contains geometrical datasets (e.g., incidence/azimuth angle, masks, etc.)

<div class="alert alert-block alert-info">
    <b>Note:</b> If you plan to use one or more ARIA GUNW correction layers — such as <b>troposphere</b>, <b>ionosphere</b>, or <b>solid Earth tides</b> — run <b>Section 1.4.2</b> <code>prep_aria.py</code> command in the second cell below.
</div>

#### 1.4.1 Use `smallbaselineApp.py` to generate MintPy stacks

In [ ]:
# Only loads the ifgram
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep load_data'
process = subprocess.run(command, shell=True)
print('Mintpy input files:')
[x for x in os.listdir('inputs') if x.endswith('.h5')]

#### 1.4.2 Use `prep_aria.py` to generate MintPy stacks, including optional corrections

In [ ]:
# Get paths from MintPy config file content
stack_dir = config_file_content['mintpy.load.unwFile'].split('/unwrapStack.vrt')[0]
dem_f = config_file_content['mintpy.load.demFile']
incAngle_f = config_file_content['mintpy.load.incAngleFile']
azAngle_f = config_file_content['mintpy.load.azAngleFile']

In [ ]:
# Set optional correction file paths if enabled and file exists
solidearthtides_f = None
ionosphere_f = None
troposphere_f = None

# Solid Earth Tides correction
if site_info.get('do_SET') == 'True':
    path = os.path.join(stack_dir, 'setStack.vrt')
    if os.path.isfile(path):
        solidearthtides_f = path
        print(f"Found: {solidearthtides_f}")
    else:
        print(f"File Not Found: {path}")

# Ionosphere correction
if site_info.get('do_iono') == 'True':
    path = os.path.join(stack_dir, 'ionoStack.vrt')
    if os.path.isfile(path):
        ionosphere_f = path
        print(f"Found: {ionosphere_f}")
    else:
        print(f"File Not Found: {path}")

# Troposphere correction
if site_info.get('do_tropo') == 'True' and site_info.get('tropo_model') == 'HRRR':
    path = os.path.join(stack_dir, 'troposphereTotal', 'HRRRStack.vrt')
    if os.path.isfile(path):
        troposphere_f = path
        print(f"Found: {troposphere_f}")
    else:
        print(f"File Not Found: {path}")


In [ ]:
command = (f"prep_aria.py -s {stack_dir} -d {dem_f} "
           f"-i {incAngle_f} -a {azAngle_f} -w {mask_file} "
           f"--set {solidearthtides_f} --tropo {troposphere_f} --iono {ionosphere_f}")
process = subprocess.run(command, shell=True)

print('Mintpy input files:')
[x for x in os.listdir('inputs') if x.endswith('.h5')]